# Create composite figures of UAV, BC, BA, BA conditions of the same fitted model.

In [1]:
# pip install pypdf
from pypdf import PdfReader, PdfWriter, PageObject, Transformation

PT_PER_IN = 72.0

def _w_h(page):
    return float(page.mediabox.width), float(page.mediabox.height)

def make_2x2_like_latex(
    pdf_paths,
    out_path="panel_2x2.pdf",
    fig_width_in=7.5,          # matches your LaTeX: 2 * 3.75in
    col_gap_pt=3.0,            # you had no \hspace; keep ~0 (LaTeX default is just line break)
    row_gap_pt=3.0,            # \smallskip ≈ 3pt
    align="top",               # "top" (usually matches LaTeX visually), or "bottom"/"center"
):
    """
    Creates a *tight* 1-page PDF consisting of 4 one-page PDFs arranged:
        [0] [1]
        [2] [3]
    mimicking LaTeX:
        subfigure width = 3.75in, includegraphics width=\linewidth
        \smallskip between rows
    """
    assert len(pdf_paths) == 4, "Need exactly 4 PDFs."

    # Read first pages
    src = []
    for p in pdf_paths:
        r = PdfReader(p)
        if not r.pages:
            raise ValueError(f"{p} has no pages.")
        src.append(r.pages[0])

    fig_w_pt = fig_width_in * PT_PER_IN
    col_w_pt = (fig_w_pt - col_gap_pt) / 2.0

    # LaTeX behavior: scale each panel by width to fill its subfigure box
    scaled_sizes = []
    scales = []
    for pg in src:
        sw, sh = _w_h(pg)
        s = col_w_pt / sw
        scales.append(s)
        scaled_sizes.append((sw * s, sh * s))

    # Row heights are max of the two panels in that row (since they share a row baseline)
    row1_h = max(scaled_sizes[0][1], scaled_sizes[1][1])
    row2_h = max(scaled_sizes[2][1], scaled_sizes[3][1])

    # Tight page size (content-only)
    out_w = fig_w_pt
    out_h = row1_h + row_gap_pt + row2_h

    dst = PageObject.create_blank_page(width=out_w, height=out_h)

    # Helper to compute y placement within a row given alignment
    def y_in_row(row_base_y, row_h, panel_h):
        if align == "top":
            return row_base_y + (row_h - panel_h)
        elif align == "center":
            return row_base_y + (row_h - panel_h) / 2.0
        elif align == "bottom":
            return row_base_y
        else:
            raise ValueError("align must be 'top', 'center', or 'bottom'.")

    # Coordinates: bottom row starts at y=0; top row above it
    row2_base = 0.0
    row1_base = row2_h + row_gap_pt

    # X positions for left/right columns
    xL = 0.0
    xR = col_w_pt + col_gap_pt

    placements = [
        # (index, x, row_base, row_h)
        (0, xL, row1_base, row1_h),
        (1, xR, row1_base, row1_h),
        (2, xL, row2_base, row2_h),
        (3, xR, row2_base, row2_h),
    ]

    for i, x, row_base, row_h in placements:
        pg = src[i]
        s = scales[i]
        pw, ph = scaled_sizes[i]
        y = y_in_row(row_base, row_h, ph)

        t = Transformation().scale(s, s).translate(x, y)
        dst.merge_transformed_page(pg, t)

    w = PdfWriter()
    w.add_page(dst)
    with open(out_path, "wb") as f:
        w.write(f)

    return out_path


In [2]:
for model_name in ["MA","MS","PM","exp-GaussianLaplace-MA","exp-GaussianLaplace-PM"]:
    pdfs = ["plots/"+model_name+"-"+cond+".pdf" for cond in ["UAV","BC","BV","BA"]]
    out = make_2x2_like_latex(
        pdf_paths=pdfs,
        out_path="plots/"+model_name+".pdf",
        fig_width_in=7.5,   # 2 * 3.75in
        col_gap_pt=5.0,    
        row_gap_pt=5.0,   
        align="bottom"        
    )
    print("Wrote:", out)

Wrote: plots/MA.pdf
Wrote: plots/MS.pdf
Wrote: plots/PM.pdf
Wrote: plots/exp-GaussianLaplace-MA.pdf
Wrote: plots/exp-GaussianLaplace-PM.pdf


# Rename figures according to their sequence in the manuscript.

The code requires installing GhostScript from https://www.ghostscript.com/releases/gsdnld.html. 

In [3]:
import shutil
import subprocess
from pathlib import Path
import os

# -------- configuration --------

PLOTS_DIR = Path("plots")

# Ordered list of supplementary figures (S1, S2, S3, ...)
# IMPORTANT: order matters
MAIN_FIGURES = [
    "TaskDesign.pdf",
    "Const-SingleGaussian_rescaleaud1.pdf",
    "ModelOverview.pdf",
    "Semiparam_FittedParams.pdf",
    "Semiparam_FittedRespDistr.pdf",
    "PM-BCAV_maintext.pdf",
    "UJoint_ModelSelection_BIC.pdf",
    "Exp-GaussianLaplace.pdf",
    "SemiparamIndv_ModelSelection_BIC.pdf",
    "exp-GaussianLaplace-PM-UAV.pdf",
    "exp-GaussianLaplace-PM-BCAV_maintext.pdf",
    "Exp-GaussianLaplace-PM_Individual_example.pdf",
    "SensoryNoisePriorParamFamilies.pdf",
    "UAV_ModelRecovery.pdf",
    "UAV_Exp-GaussianLaplace_ParamRecovery.pdf"
]
SUPP_FIGURES = [
    # Alternative lapse distributions
    "Exp-GaussianLaplace_nolapse.pdf",
    "Exp-GaussianLaplace_Gaussianlapse.pdf",

    # Model selection figures
    "UJoint_ModelSelection_vanilla.pdf",
    "UJoint_ModelSelection_BIC_full.pdf",
    "SemiparamIndv_ModelSelection_BIC_full.pdf",

    # Unisensory parametric response distributions
    "Const-SingleGaussian.pdf",
    "Exp-SingleGaussian.pdf",
    "Const-GaussianLaplace.pdf",
    "Exp-TwoGaussians.pdf",

    # Lifted-semiparametric (MS)
    "MS.pdf",

    # Lifted-semiparametric (MA)
    "MA.pdf",

    # Lifted-semiparametric (PM)
    "PM.pdf",

    # All-tasks parametric (MA)
    "exp-GaussianLaplace-MA.pdf",

    # All-tasks parametric (PM)
    "exp-GaussianLaplace-PM.pdf",

    # Individual participant figures
    "Exp-GaussianLaplace_Individualmean.pdf",
    "Exp-GaussianLaplace_IndividualSD.pdf",
    "Exp-GaussianLaplace-PM-UAV_Individualmean.pdf",
    "exp-GaussianLaplace-PM-UAV_IndividualSD.pdf",
    "Exp-GaussianLaplace-PM-BC_Individual.pdf",
    "Exp-GaussianLaplace-PM-BV_Individual.pdf",
    "Exp-GaussianLaplace-PM-BA_Individual.pdf",
]
MAIN_FIGURES = ["plots/"+fig for fig in MAIN_FIGURES]
SUPP_FIGURES = ["plots/"+fig for fig in SUPP_FIGURES]


def find_ghostscript_exe() -> str:
    """
    Return a working Ghostscript command.
    On Windows, prefer gswin64c/gswin32c; on Unix, prefer gs.
    Raises RuntimeError with actionable instructions if not found.
    """
    # 1) If on PATH
    for cmd in ("gswin64c", "gswin32c", "gs"):
        p = shutil.which(cmd)
        if p:
            return p

    # 2) Common Windows install locations
    if os.name == "nt":
        candidates = []
        for base in (Path(r"C:\Program Files"), Path(r"C:\Program Files (x86)")):
            if base.exists():
                # Ghostscript folders look like: Ghostscript\gs10.02.1\bin\gswin64c.exe
                gs_root = base / "gs"
                if gs_root.exists():
                    candidates.extend(gs_root.glob(r"gs*\bin\gswin64c.exe"))
                    candidates.extend(gs_root.glob(r"gs*\bin\gswin32c.exe"))
                ghostscript_root = base / "Ghostscript"
                if ghostscript_root.exists():
                    candidates.extend(ghostscript_root.glob(r"gs*\bin\gswin64c.exe"))
                    candidates.extend(ghostscript_root.glob(r"gs*\bin\gswin32c.exe"))

        for c in candidates:
            if c.exists():
                return str(c)

    raise RuntimeError(
        "Ghostscript was not found.\n"
        "Install Ghostscript and ensure its command is on PATH.\n"
        "Windows: install from ghostscript.com, then make sure gswin64c.exe is on PATH.\n"
        "macOS: brew install ghostscript\n"
        "Ubuntu/Debian: sudo apt install ghostscript"
    )

def pdf_to_eps(gs_exe: str, pdf_path: Path, eps_path: Path):
    """
    Convert PDF to EPS using Ghostscript (vector-safe).
    -dEPSCrop uses the PDF's bounding box for tight EPS.
    """
    cmd = [
        gs_exe,
        "-dSAFER",
        "-dBATCH",
        "-dNOPAUSE",
        "-dEPSCrop",
        "-sDEVICE=eps2write",
        f"-sOutputFile={str(eps_path)}",
        str(pdf_path),
    ]
    subprocess.run(cmd, check=True)
    
def process_main_figures():
    gs_exe = find_ghostscript_exe()
    print(f"Using Ghostscript: {gs_exe}")

    for idx, src in enumerate(MAIN_FIGURES, start=1):
        src_pdf = Path(src)
        if not src_pdf.exists():
            raise FileNotFoundError(f"Missing input PDF: {src_pdf}")

        renamed_pdf = PLOTS_DIR / f"fig{idx}.pdf"
        eps_path    = PLOTS_DIR / f"fig{idx}.eps"

        # Copy/rename to S{idx}_fig.pdf
        shutil.copyfile(src_pdf, renamed_pdf)

        # Convert to EPS
        pdf_to_eps(gs_exe, renamed_pdf, eps_path)

        print(f"✓ Fig {idx}: {src_pdf.name} -> {renamed_pdf.name} -> {eps_path.name}")

def process_supplementary_figures():
    gs_exe = find_ghostscript_exe()
    print(f"Using Ghostscript: {gs_exe}")

    for idx, src in enumerate(SUPP_FIGURES, start=1):
        src_pdf = Path(src)
        if not src_pdf.exists():
            raise FileNotFoundError(f"Missing input PDF: {src_pdf}")

        renamed_pdf = PLOTS_DIR / f"S{idx}_fig.pdf"
        eps_path    = PLOTS_DIR / f"S{idx}_fig.eps"

        # Copy/rename to S{idx}_fig.pdf
        shutil.copyfile(src_pdf, renamed_pdf)

        # Convert to EPS
        pdf_to_eps(gs_exe, renamed_pdf, eps_path)

        print(f"✓ Fig S{idx}: {src_pdf.name} -> {renamed_pdf.name} -> {eps_path.name}")

if __name__ == "__main__":
    process_main_figures()
    print()
    process_supplementary_figures()



Using Ghostscript: C:\Program Files\gs\gs10.06.0\bin\gswin64c.EXE
✓ Fig 1: TaskDesign.pdf -> fig1.pdf -> fig1.eps
✓ Fig 2: Const-SingleGaussian_rescaleaud1.pdf -> fig2.pdf -> fig2.eps
✓ Fig 3: ModelOverview.pdf -> fig3.pdf -> fig3.eps
✓ Fig 4: Semiparam_FittedParams.pdf -> fig4.pdf -> fig4.eps
✓ Fig 5: Semiparam_FittedRespDistr.pdf -> fig5.pdf -> fig5.eps
✓ Fig 6: PM-BCAV_maintext.pdf -> fig6.pdf -> fig6.eps
✓ Fig 7: UJoint_ModelSelection_BIC.pdf -> fig7.pdf -> fig7.eps
✓ Fig 8: Exp-GaussianLaplace.pdf -> fig8.pdf -> fig8.eps
✓ Fig 9: SemiparamIndv_ModelSelection_BIC.pdf -> fig9.pdf -> fig9.eps
✓ Fig 10: exp-GaussianLaplace-PM-UAV.pdf -> fig10.pdf -> fig10.eps
✓ Fig 11: exp-GaussianLaplace-PM-BCAV_maintext.pdf -> fig11.pdf -> fig11.eps
✓ Fig 12: Exp-GaussianLaplace-PM_Individual_example.pdf -> fig12.pdf -> fig12.eps
✓ Fig 13: SensoryNoisePriorParamFamilies.pdf -> fig13.pdf -> fig13.eps
✓ Fig 14: UAV_ModelRecovery.pdf -> fig14.pdf -> fig14.eps
✓ Fig 15: UAV_Exp-GaussianLaplace_ParamReco